In [1]:
import pymc3 as pm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az
import re

In [2]:
full = pd.read_csv("Data/full_data.csv")
full.head()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,All_NBA_Pos,All_NBA_Team,All_NBA_Pts_Won,All_NBA_Pts_Max,All_NBA_Share,All_NBA_1st_Team_Votes,All_NBA_2nd_Team_Votes,All_NBA_3rd_Team_Votes,year,All_NBA_Boolean
0,Kareem Abdul-Jabbar*,C,41,LAL,74,1695,12.9,0.511,0.005,0.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
1,Mark Acres,C,26,BOS,62,632,8.2,0.507,0.009,0.421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
2,Michael Adams,PG,26,DEN,77,2787,17.5,0.567,0.431,0.363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
3,Mark Alarie,PF,25,WSB,74,1141,13.3,0.531,0.088,0.202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False
4,Randy Allen,SG,24,SAC,7,43,6.9,0.428,0.053,0.105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-1989,False


In [3]:
## Remove All ALL Stats except boolean All_Stars
remove_all_star = full.loc[:, ~full.columns.str.startswith('All_')]
remove_all_star["All_NBA_Boolean"] = full["All_NBA_Boolean"]
remove_all_star.head()

/var/folders/pn/dgy7ckd90nl7mlj6g6rc_1kw0000gn/T/ipykernel_9473/1871535054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_all_star["All_NBA_Boolean"] = full["All_NBA_Boolean"]


,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player_id,year,All_NBA_Boolean
0,Kareem Abdul-Jabbar*,C,41,LAL,74,1695,12.9,0.511,0.005,0.250,...,2.0,2.9,0.082,-1.9,-0.2,-2.1,0.0,abdulka01,1988-1989,False
1,Mark Acres,C,26,BOS,62,632,8.2,0.507,0.009,0.421,...,0.5,0.9,0.066,-3.0,-0.4,-3.4,-0.2,acresma01,1988-1989,False
2,Michael Adams,PG,26,DEN,77,2787,17.5,0.567,0.431,0.363,...,2.9,8.3,0.143,2.7,0.2,2.9,3.4,adamsmi01,1988-1989,False
3,Mark Alarie,PF,25,WSB,74,1141,13.3,0.531,0.088,0.202,...,1.0,2.2,0.093,-1.1,-0.8,-2.0,0.0,alarima01,1988-1989,False
4,Randy Allen,SG,24,SAC,7,43,6.9,0.428,0.053,0.105,...,0.0,0.0,-0.035,-5.5,-1.7,-7.2,-0.1,allenra01,1988-1989,False


In [4]:
remove_nan = remove_all_star.dropna()
remove_nan

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,player_id,year,All_NBA_Boolean
0,Kareem Abdul-Jabbar*,C,41,LAL,74,1695,12.9,0.511,0.005,0.250,...,2.0,2.9,0.082,-1.9,-0.2,-2.1,0.0,abdulka01,1988-1989,False
1,Mark Acres,C,26,BOS,62,632,8.2,0.507,0.009,0.421,...,0.5,0.9,0.066,-3.0,-0.4,-3.4,-0.2,acresma01,1988-1989,False
2,Michael Adams,PG,26,DEN,77,2787,17.5,0.567,0.431,0.363,...,2.9,8.3,0.143,2.7,0.2,2.9,3.4,adamsmi01,1988-1989,False
3,Mark Alarie,PF,25,WSB,74,1141,13.3,0.531,0.088,0.202,...,1.0,2.2,0.093,-1.1,-0.8,-2.0,0.0,alarima01,1988-1989,False
4,Randy Allen,SG,24,SAC,7,43,6.9,0.428,0.053,0.105,...,0.0,0.0,-0.035,-5.5,-1.7,-7.2,-0.1,allenra01,1988-1989,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15438,Tremont Waters,PG,24,TOT,3,50,5.4,0.357,0.643,0.000,...,0.1,0.0,-0.042,-7.8,0.1,-7.7,-0.1,watertr01,2021-2022,False
15439,Derrick White,SG,27,TOT,75,2199,14.8,0.549,0.461,0.294,...,2.3,5.1,0.111,0.4,0.6,0.9,1.6,whitede01,2021-2022,False
15440,Justise Winslow,SF,25,TOT,48,774,13.1,0.479,0.257,0.257,...,1.1,0.8,0.048,-2.7,1.0,-1.7,0.1,winslju01,2021-2022,False
15441,Moses Wright,PF,23,TOT,4,14,18.3,0.470,0.250,0.750,...,0.0,0.1,0.180,1.1,0.4,1.4,0.0,wrighmo01,2021-2022,False


In [5]:
pd.factorize(remove_nan.year)

(array([ 0,  0,  0, ..., 33, 33, 33]),
 Index(['1988-1989', '1989-1990', '1990-1991', '1991-1992', '1992-1993',
        '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998',
        '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003',
        '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008',
        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013',
        '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018',
        '2018-2019', '2019-2020', '2020-2021', '2021-2022'],
       dtype='object'))

In [6]:
season_idxs, seasons = pd.factorize(remove_nan.year)
coords = {
    "season": seasons,
    "obs_id": np.arange(len(season_idxs)),
}
coords

{'season': Index(['1988-1989', '1989-1990', '1990-1991', '1991-1992', '1992-1993',
        '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998',
        '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003',
        '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008',
        '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013',
        '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018',
        '2018-2019', '2019-2020', '2020-2021', '2021-2022'],
       dtype='object'),
 'obs_id': array([    0,     1,     2, ..., 15379, 15380, 15381])}

In [7]:
season_idxs

array([ 0,  0,  0, ..., 33, 33, 33])

In [8]:
seasons

Index(['1988-1989', '1989-1990', '1990-1991', '1991-1992', '1992-1993',
       '1993-1994', '1994-1995', '1995-1996', '1996-1997', '1997-1998',
       '1998-1999', '1999-2000', '2000-2001', '2001-2002', '2002-2003',
       '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008',
       '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013',
       '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018',
       '2018-2019', '2019-2020', '2020-2021', '2021-2022'],
      dtype='object')

In [9]:
import theano.tensor as T
def logistic(l):
    return 1 / (1 + T.exp(-l))

In [10]:
remove_nan.columns[1:-2]

Index(['Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'player_id'],
      dtype='object')

In [11]:
numeric_col = pd.DataFrame(remove_nan.dtypes[:-1]).reset_index()
numeric_col = numeric_col[numeric_col[0] != 'object']
numeric_col.columns = ['name', 'type']
numeric_col

,name,type
2,Age,int64
4,G,int64
5,MP,int64
6,PER,float64
7,TS%,float64
8,3PAr,float64
9,FTr,float64
10,ORB%,float64
11,DRB%,float64
12,TRB%,float64


In [12]:
for i,k in enumerate(numeric_col.name[0:5]):
    print(i,k)

0 Age
1 G
2 MP
3 PER
4 TS%


In [13]:
master_df = []
for i,col in enumerate(numeric_col.name):
    with pm.Model(coords=coords) as hierarchical_model:
        season_idx = pm.Data("season_idx", season_idxs, dims="obs_id")
        # Hyperpriors for group nodes
        mu_a = pm.Normal("mu_a", mu=0.0, sigma=10000)
        sigma_a = pm.HalfNormal("sigma_a", 5.0)
        mu_b = pm.Normal("mu_b", mu=0.0, sigma=10000)
        sigma_b = pm.HalfNormal("sigma_b", 5.0)


        # Intercept for each county, distributed around group mean mu_a
        # Above we just set mu and sd to a fixed value while here we
        # plug in a common group distribution for all a and b (which are
        # vectors of length n_counties).
        a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="season")
        # effect difference between basement and floor level
        b = pm.Normal("b", mu=mu_b, sigma=sigma_b, dims="season")

        nba_est = a[season_idx] + b[season_idx] * remove_nan[col].values

        # Data likelihood
        nba_like = pm.Bernoulli(
            "nba_like",
            # p=logistic(nba_est),
            logit_p = nba_est,
            observed=remove_nan.All_NBA_Boolean.astype(int).values
        )
    # with hierarchical_model:
        hierarchical_trace = pm.sample(100, tune=100, target_accept=0.6, return_inferencedata=True)
        df = pm.summary(hierarchical_trace).reset_index().rename(columns={"index":"distribution"})
        df['predictor'] = [col]*len(df)
        master_df.append(df)

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a, sigma_b, mu_b, sigma_a, mu_a]


Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 27 seconds.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.1690615722071727, but should be close to 0.6. Try to increase the number of tuning steps.
There were 57 divergences after tuning. Increase `target_accept` or reparameterize.
There were 26 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7932051734404666, but should be close to 0.6. Try to increase the number of tuning steps.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.3767886595057534, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 20 seconds.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.1550251609550568, but should be close to 0.6. Try to increase the number of tuning steps.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 0.10835404370997237, but should be close to 0.6. Try to increase the number of tuning steps.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 5.370599705889921e-76, but should be close to 0.6. Try to increase the number of tuning steps.
There were 61 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7015222723059408, but should be close to 0.6. Try to inc

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 13 seconds.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 0.0, but should be close to 0.6. Try to increase the number of tuning steps.
There were 97 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.04733110010879939, but should be close to 0.6. Try to increase the number of tuning steps.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 0.0, but should be close to 0.6. Try to increase the number of tuning steps.
There were 11 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.1782877827853364, but should be close to 0.6. Try to increase the number of tuning steps.

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 19 seconds.
The acceptance probability does not match the target. It is 0.7915643349827229, but should be close to 0.6. Try to increase the number of tuning steps.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7883802985820698, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7877409434630797, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7543959038923765, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some parameters.
Only 100 samples in chain.
Auto-assigning NUTS samp

...

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 20 seconds.
The acceptance probability does not match the target. It is 0.7893902570844769, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.7522524821273222, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9595845142541733, but should be close to 0.6. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8480619098561403, but should be close to 0.6. Try to increase the number of tuning steps.
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a, sigma_b, mu_b, sigma_a, mu_a]


Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 16 seconds.
There were 28 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7442613773967852, but should be close to 0.6. Try to increase the number of tuning steps.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7424530561037305, but should be close to 0.6. Try to increase the number of tuning steps.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7387857651346477, but should be close to 0.6. Try to increase the number of tuning steps.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7216024797379552, but should be close to 0.6. Try t

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 23 seconds.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8991297950235725, but should be close to 0.6. Try to increase the number of tuning steps.
The chain contains only diverging samples. The model is probably misspecified.
The acceptance probability does not match the target. It is 0.2577224141151705, but should be close to 0.6. Try to increase the number of tuning steps.
There were 52 divergences after tuning. Increase `target_accept` or reparameterize.
There were 98 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.3375260674879387, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samp

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 15 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7254351661896389, but should be close to 0.6. Try to increase the number of tuning steps.
There were 94 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.279637504393131, but should be close to 0.6. Try to increase the number of tuning steps.
There were 23 divergences after tuning. Increase `target_accept` or reparameterize.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7445331974317128, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective sa

Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 15 seconds.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7609068046198878, but should be close to 0.6. Try to increase the number of tuning steps.
There were 16 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7589595830328736, but should be close to 0.6. Try to increase the number of tuning steps.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7106684565394168, but should be close to 0.6. Try to increase the number of tuning steps.
There were 47 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective

In [14]:
final_master_df = pd.concat(master_df)
final_master_df.head(10)

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor
0,mu_a,-1.010,0.314,-1.651,-0.781,0.145,0.110,5.0,12.0,3.45,Age
1,mu_b,-0.088,0.012,-0.099,-0.063,0.006,0.004,5.0,12.0,3.06,Age
2,a[1988-1989],-0.985,0.318,-1.625,-0.711,0.147,0.112,5.0,12.0,3.16,Age
3,a[1989-1990],-1.007,0.310,-1.655,-0.757,0.143,0.108,5.0,11.0,3.39,Age
4,a[1990-1991],-1.010,0.328,-1.660,-0.730,0.148,0.111,5.0,12.0,2.93,Age
5,a[1991-1992],-0.942,0.350,-1.619,-0.666,0.161,0.122,5.0,12.0,3.10,Age
6,a[1992-1993],-1.032,0.298,-1.716,-0.710,0.135,0.102,6.0,11.0,2.58,Age
7,a[1993-1994],-0.983,0.323,-1.625,-0.741,0.149,0.112,5.0,12.0,3.01,Age
8,a[1994-1995],-0.984,0.321,-1.734,-0.770,0.148,0.112,6.0,12.0,2.53,Age
9,a[1995-1996],-1.000,0.323,-1.612,-0.758,0.147,0.111,5.0,12.0,3.34,Age


In [15]:
final_master_df['significant'] = ((final_master_df["hdi_3%"] < 0) & (final_master_df["hdi_97%"] > 0)) == False
final_master_df['typeDist'] = np.where(final_master_df['distribution'].str.contains('b'), 'beta', 'intercept')
final_master_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor,significant,typeDist
0,mu_a,-1.010,0.314,-1.651,-0.781,0.145,0.110,5.0,12.0,3.45,Age,True,intercept
1,mu_b,-0.088,0.012,-0.099,-0.063,0.006,0.004,5.0,12.0,3.06,Age,True,beta
2,a[1988-1989],-0.985,0.318,-1.625,-0.711,0.147,0.112,5.0,12.0,3.16,Age,True,intercept
3,a[1989-1990],-1.007,0.310,-1.655,-0.757,0.143,0.108,5.0,11.0,3.39,Age,True,intercept
4,a[1990-1991],-1.010,0.328,-1.660,-0.730,0.148,0.111,5.0,12.0,2.93,Age,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,b[2019-2020],1.847,0.150,1.577,2.097,0.046,0.034,10.0,23.0,1.34,VORP,True,beta
68,b[2020-2021],1.824,0.127,1.629,2.036,0.028,0.020,22.0,147.0,1.14,VORP,True,beta
69,b[2021-2022],1.790,0.133,1.552,1.999,0.041,0.030,11.0,24.0,1.33,VORP,True,beta
70,sigma_a,0.384,0.123,0.216,0.582,0.041,0.030,8.0,26.0,1.52,VORP,True,intercept


In [16]:
season_final_master_df = final_master_df[final_master_df["distribution"].str.contains('\[')]
season_final_master_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor,significant,typeDist
2,a[1988-1989],-0.985,0.318,-1.625,-0.711,0.147,0.112,5.0,12.0,3.16,Age,True,intercept
3,a[1989-1990],-1.007,0.310,-1.655,-0.757,0.143,0.108,5.0,11.0,3.39,Age,True,intercept
4,a[1990-1991],-1.010,0.328,-1.660,-0.730,0.148,0.111,5.0,12.0,2.93,Age,True,intercept
5,a[1991-1992],-0.942,0.350,-1.619,-0.666,0.161,0.122,5.0,12.0,3.10,Age,True,intercept
6,a[1992-1993],-1.032,0.298,-1.716,-0.710,0.135,0.102,6.0,11.0,2.58,Age,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,b[2017-2018],1.678,0.113,1.506,1.913,0.017,0.012,38.0,175.0,1.10,VORP,True,beta
66,b[2018-2019],1.639,0.134,1.381,1.860,0.042,0.030,11.0,16.0,1.33,VORP,True,beta
67,b[2019-2020],1.847,0.150,1.577,2.097,0.046,0.034,10.0,23.0,1.34,VORP,True,beta
68,b[2020-2021],1.824,0.127,1.629,2.036,0.028,0.020,22.0,147.0,1.14,VORP,True,beta


In [17]:
final_master_df.groupby(['predictor', 'typeDist']).sum()['significant']

predictor  typeDist 
3PAr       beta         36
           intercept    36
AST%       beta         36
           intercept    36
Age        beta         36
           intercept    36
BLK%       beta         35
           intercept    36
BPM        beta         35
           intercept    36
DBPM       beta         36
           intercept    36
DRB%       beta         36
           intercept    36
DWS        beta         36
           intercept    36
FTr        beta         36
           intercept    36
G          beta         36
           intercept    36
MP         beta         25
           intercept     5
OBPM       beta         35
           intercept    36
ORB%       beta          1
           intercept    36
OWS        beta         36
           intercept    36
PER        beta         36
           intercept    36
STL%       beta         36
           intercept    36
TOV%       beta         36
           intercept    36
TRB%       beta         36
           intercept    36
TS%    

In [18]:
season_final_master_df.groupby(['predictor', 'typeDist']).sum()['significant']

predictor  typeDist 
3PAr       beta         34
           intercept    34
AST%       beta         34
           intercept    34
Age        beta         34
           intercept    34
BLK%       beta         33
           intercept    34
BPM        beta         33
           intercept    34
DBPM       beta         34
           intercept    34
DRB%       beta         34
           intercept    34
DWS        beta         34
           intercept    34
FTr        beta         34
           intercept    34
G          beta         34
           intercept    34
MP         beta         24
           intercept     4
OBPM       beta         33
           intercept    34
ORB%       beta          0
           intercept    34
OWS        beta         34
           intercept    34
PER        beta         34
           intercept    34
STL%       beta         34
           intercept    34
TOV%       beta         34
           intercept    34
TRB%       beta         34
           intercept    34
TS%    

In [25]:
season_final_master_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,predictor,significant,typeDist
2,a[1988-1989],-0.985,0.318,-1.625,-0.711,0.147,0.112,5.0,12.0,3.16,Age,True,intercept
3,a[1989-1990],-1.007,0.310,-1.655,-0.757,0.143,0.108,5.0,11.0,3.39,Age,True,intercept
4,a[1990-1991],-1.010,0.328,-1.660,-0.730,0.148,0.111,5.0,12.0,2.93,Age,True,intercept
5,a[1991-1992],-0.942,0.350,-1.619,-0.666,0.161,0.122,5.0,12.0,3.10,Age,True,intercept
6,a[1992-1993],-1.032,0.298,-1.716,-0.710,0.135,0.102,6.0,11.0,2.58,Age,True,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,b[2017-2018],1.678,0.113,1.506,1.913,0.017,0.012,38.0,175.0,1.10,VORP,True,beta
66,b[2018-2019],1.639,0.134,1.381,1.860,0.042,0.030,11.0,16.0,1.33,VORP,True,beta
67,b[2019-2020],1.847,0.150,1.577,2.097,0.046,0.034,10.0,23.0,1.34,VORP,True,beta
68,b[2020-2021],1.824,0.127,1.629,2.036,0.028,0.020,22.0,147.0,1.14,VORP,True,beta


In [19]:
len(np.unique(remove_nan.year))

34

In [26]:
with pm.Model(coords=coords) as hierarchical_model:
    season_idx = pm.Data("season_idx", season_idxs, dims="obs_id")
    # Hyperpriors for group nodes
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=10000)
    sigma_a = pm.HalfNormal("sigma_a", 5.0)

    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="season")

    nba_est = a[season_idx]
    for i,col in enumerate(numeric_col.name):
        mu = pm.Normal("mu_b{}".format(i), mu=0.0, sigma=10000)
        sigma = pm.HalfNormal("sigma_b{}".format(i), 5.0)
        var = pm.Normal("b{}".format(i), mu=mu, sigma=sigma, dims="season")
        nba_est = nba_est + var[season_idx] * remove_nan[col].values

    # Data likelihood
    nba_like = pm.Bernoulli(
        "nba_like",
        # p=logistic(nba_est),
        logit_p = nba_est,
        observed=remove_nan.All_NBA_Boolean.astype(int).values
    )

In [27]:
with hierarchical_model:
    hierarchical_trace = pm.sample(100, tune=100, target_accept=0.6, return_inferencedata=True)

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b22, sigma_b22, mu_b22, b21, sigma_b21, mu_b21, b20, sigma_b20, mu_b20, b19, sigma_b19, mu_b19, b18, sigma_b18, mu_b18, b17, sigma_b17, mu_b17, b16, sigma_b16, mu_b16, b15, sigma_b15, mu_b15, b14, sigma_b14, mu_b14, b13, sigma_b13, mu_b13, b12, sigma_b12, mu_b12, b11, sigma_b11, mu_b11, b10, sigma_b10, mu_b10, b9, sigma_b9, mu_b9, b8, sigma_b8, mu_b8, b7, sigma_b7, mu_b7, b6, sigma_b6, mu_b6, b5, sigma_b5, mu_b5, b4, sigma_b4, mu_b4, b3, sigma_b3, mu_b3, b2, sigma_b2, mu_b2, b1, sigma_b1, mu_b1, b0, sigma_b0, mu_b0, a, sigma_a, mu_a]


Sampling 4 chains for 100 tune and 100 draw iterations (400 + 400 draws total) took 617 seconds.
There were 76 divergences after tuning. Increase `target_accept` or reparameterize.
There were 73 divergences after tuning. Increase `target_accept` or reparameterize.
There were 42 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.7624237698099443, but should be close to 0.6. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 69 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.3322758310828781, but should be close to 0.6. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The number of effective samples is smaller than 10% for some param

In [23]:
hierarchical_trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data
	> constant_data

In [30]:
all_predictor_df = pm.summary(hierarchical_trace).reset_index().rename(columns={"index":"distribution"})
all_predictor_df['significant'] = ((all_predictor_df["hdi_3%"] < 0) & (all_predictor_df["hdi_97%"] > 0)) == False
all_predictor_df['typeDist'] = np.where(all_predictor_df['distribution'].str.contains('b'), 'beta', 'intercept')
all_predictor_df

,distribution,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,significant,typeDist
0,mu_a,-0.527,0.367,-0.960,0.022,0.172,0.131,5.0,12.0,3.70,False,intercept
1,a[1988-1989],-0.158,0.458,-0.903,0.475,0.218,0.166,5.0,14.0,3.33,False,intercept
2,a[1989-1990],0.711,0.259,0.388,1.290,0.123,0.093,5.0,12.0,3.81,True,intercept
3,a[1990-1991],0.170,0.567,-0.917,0.732,0.270,0.205,5.0,12.0,3.21,False,intercept
4,a[1991-1992],0.062,0.394,-0.372,0.673,0.187,0.142,5.0,6.0,2.99,False,intercept
...,...,...,...,...,...,...,...,...,...,...,...,...
859,sigma_b18,5.671,1.759,3.418,8.915,0.767,0.576,6.0,12.0,2.14,True,beta
860,sigma_b19,5.531,1.426,2.359,8.049,0.606,0.454,6.0,12.0,2.68,True,beta
861,sigma_b20,4.141,2.761,1.443,9.440,1.241,0.935,5.0,12.0,2.26,True,beta
862,sigma_b21,2.602,0.696,1.144,3.350,0.309,0.233,5.0,12.0,2.61,True,beta
